# The GRAD-GPAD framework 🗿 
    
    ➡️ Visualization
---

The `gradgpad` framework povides *Python* tooling to create novel visualization graphs to ease research pipeline and fair comparision on face-PAD topic. 

This tutorial is a detailed description of main visualizations available in `gradgpad`. Additionaly, thanks to `ipywidgets` standard module we can interact with visualizations. 


## Table of Contents 👩‍💻
1. Software Design ⚖️
2. Installation 💻
3. Import gradgpad
4. Histogram Plotter
5. DET Plotter
6. PAD Radar Plotter
7. Bias Percentiles Plotter

---
### <span style='color :blue' > 1. Software Design ⚖️ </span>  

<img src="images/gradgpad_detailed_architecture_highlight_visualization.jpeg" align="left" width="600" height="800" />

### <span style='color :blue' > 2. Installation 💻 </span>  


In [ ]:
!pip install -U gradgpad

### <span style='color :blue' > 3. Import gradgpad </span>  

In [ ]:
import os
from ipywidgets import interact, Layout, FloatSlider, Dropdown

from gradgpad import (
    PadRadarPaiPlotter,
    PadRadarProtocolPlotter,
    BiasPercentilePlotter,
    HistogramPlotter,
    DetPlotter,
    WorkingPoint,
    FineGrainedPaisProvider,
    CombinedScenario,
    ResultsProvider,
    Approach,
    Protocol, 
    GifCreator,
    ScoresProvider,
    Subset,
    Metrics,
    SplitByLabelMode,
    Demographic,
    GrainedPaiMode
)

# Load Results
results = {
    "Quality": ResultsProvider.get(
        Approach.QUALITY_RBF, protocol=Protocol.GRANDTEST
    ),
    "Auxiliary": ResultsProvider.get(
        Approach.AUXILIARY, protocol=Protocol.GRANDTEST
    ),
}

all_results = {
    "Quality": ResultsProvider.all(
        Approach.QUALITY_RBF
    ),
    "Auxiliary": ResultsProvider.all(
        Approach.AUXILIARY
    ),
}

# Load Scores
scores_auxiliary_devel = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.DEVEL
)
scores_auxiliary_test = ScoresProvider.get(
    Approach.AUXILIARY, 
    Protocol.GRANDTEST, 
    Subset.TEST
)

output_path = "output"
os.makedirs(output_path, exist_ok=True)

---
### <span style='color :blue' > 4. Histogram Plotter </span>  

Use `HistogramPlotter` to calculate a plot an histogram with several configurable parameters.

In [ ]:
histogram_plotter = HistogramPlotter()
histogram_plotter.show(scores_auxiliary_test)

Save the histogram with `save` method

In [ ]:
histogram_plotter.save(f"{output_path}/my_histogram.png", scores_auxiliary_test)

You can also configure:

* Normalize the histogram with the boolean parameter `normalize`. 
* Change the title (using the `title` parameter)
* Add a limitation for y values with `y_max_value` [0-1]


In [ ]:
histogram_plotter = HistogramPlotter(
    normalize=True,
    title="My Histogram",
    y_max_value=1.4,
)
histogram_plotter.show(scores_auxiliary_test)

If you want to add a threshold, you can do it with `plot_vertical_line_on_value` parameter. Use `legend_vertical_line` to customize the legend.

In the following example, we calculate the EER threshold in `DEVEL` and show the Histogram in `TEST`

In [ ]:
metrics = Metrics(scores_auxiliary_devel, scores_auxiliary_test)
eer_th = metrics.get_eer_th(Subset.DEVEL)

histogram_plotter = HistogramPlotter(
    plot_vertical_line_on_value=eer_th, 
    legend_vertical_line="EER @ Devel"
)
histogram_plotter.show(scores_auxiliary_test)

###### <span style='color :green' > 4.1 Interactive Histogram Plotter </span>  

You can also plot different subgrups to check how is distributed the scores. 
Use the following interactive function to play with the `HistogramPlotter` class

In [ ]:
@interact(layout=Layout(width='120%', height='80px'))
def calculate_histogram(
    split_mode=SplitByLabelMode.options(),
    show_attacks=[True, False],
    normalize=[True, False],
    approach=Approach.options(),
    protocol=Protocol.grandtest_options(),
    subset=Subset.options()
):
    title = f"Histogram PAIs ({split_mode.value})"
            
    histogram_plotter = HistogramPlotter(
        title=title,
        split_by_label_mode=split_mode,
        normalize=normalize,
        exclude_labels=[-1] if show_attacks is False else None,
    )

    scores = ScoresProvider.get(
        approach, 
        protocol, 
        subset
    )
    histogram_plotter.show(scores)

---
### <span style='color :blue' > 5. DET Plotter </span>  

Use `DetPlotter` to plot DET curve (Detection Error Tradeoff) where error rates for binary classification system are represented plotting the false rejection rate vs. false acceptance rate.

In [ ]:
det_plotter = DetPlotter()
det_plotter.show(scores_auxiliary_test)

Save calculated DET curve with `save` method.

In [ ]:
det_plotter.save(f"{output_path}/my_det.png", scores_auxiliary_test)

You can also configure:

* Change the title (using the `title` parameter)
* You can also plot different subgrups performance with `split_by_label_mode`. 


In [ ]:
det_plotter = DetPlotter(
    title="Demographic Sex",
    split_by_label_mode=SplitByLabelMode.SEX
)
det_plotter.show(scores_auxiliary_test)

###### <span style='color :green' > 5.1 Interactive DET Plotter </span>  


Use the following interactive function to play with the `Det` class

In [ ]:
@interact(layout=Layout(width='120%', height='80px'))
def calculate_det_curve(
    split_mode=SplitByLabelMode.options_for_curves(),
):
    title = f"DET PAIs ({split_mode.value})"
    
    det_plotter = DetPlotter(
        title=title,
        split_by_label_mode=split_mode,
    )

    det_plotter.show(scores_auxiliary_test)

---
### <span style='color :blue' > 6. PAD Radar Plotter </span>  


The “PAD-radar” show us information related to model’s generalization and PAI behaviour.

* *PAD-Radar by PAI*: Use `PadRadarPaiPlotter` to calculate a radar graph where each vertex is the performance (APCER for a given BPCER working point) of an specific PAI.
* *PAD-Radar by protocols*: Use `PadRadarProtocolPlotter`to calculate a radar graph where each vertex is the performance (APCER for a given BPCER working point) of an specific subprotocol (e.g cross-dataset -> Replay-Mobile).


##### 6.1 PAD Radar By PAI

In [ ]:
pad_radar_pai_plotter = PadRadarPaiPlotter(
    title="My First PAD-Radar",
    working_point=WorkingPoint.BPCER_10,
    combined_scenario=CombinedScenario.PAS_II_AND_III
)
pad_radar_pai_plotter.show(results)

###### <span style='color :green' > 6.1.1 Interactive PAD radar by PAI Plotter </span>  


Let's do it this interactive. Select the working point and the PAS (Presentation Attack Scenario)

In [ ]:
@interact
def calculate_pad_radar_by_pai(
    wp=WorkingPoint.options(),
    scenario=CombinedScenario.options()
):
    title = f"PAD-Radar | APCER {wp}%".replace("WorkingPoint.", "@ ").replace("_", " ")
    
    pad_radar_pai_plotter = PadRadarPaiPlotter(
        title=title,
        working_point=wp,
        combined_scenario=scenario
    )
    pad_radar_pai_plotter.show(results)

##### 6.2. PAD Radar By Protocol

In [ ]:
pad_radar_protocol_plotter = PadRadarProtocolPlotter(
    title="PAD-Radar (Cross-Database)",
    working_point=WorkingPoint.BPCER_10,
    grained_pai_mode=GrainedPaiMode.FINE,
    protocol=Protocol.CROSS_DATASET
)
pad_radar_protocol_plotter.show(all_results)

###### <span style='color :green' > 6.2.1 Interactive PAD radar by Protocol Plotter </span>  

In [ ]:
@interact
def calculate_pad_radar_by_prtocol(
    wp=WorkingPoint.options(),
    grained_pai_mode=GrainedPaiMode.options(),
    protocol=Protocol.generalization_options()
):
    title = f"PAD-Radar {protocol.value} | APCER {wp}%".replace("WorkingPoint.", "@ ").replace("_", " ")
    pad_radar_protocol_plotter = PadRadarProtocolPlotter(
        title=title,
        working_point=wp,
        grained_pai_mode=grained_pai_mode,
        protocol=protocol
    )
    pad_radar_protocol_plotter.show(all_results)

##### 6.3. Create a dynamic PAD radar image (GIF)

Save several PAD radar (e.g different working point).

In [ ]:
saved_filenames = []
for working_point in WorkingPoint.options():
    title = f"PAD-Radar | APCER {working_point}%".replace("WorkingPoint.", "@ ").replace("_", " ")
    output_filename = f"{output_path}/pad_radar_apcer_{working_point}.png".lower()

    pad_radar_pai_plotter = PadRadarPaiPlotter(
        title=title,
        working_point=working_point,
        combined_scenario=CombinedScenario.PAS_I_AND_II
    )
    pad_radar_pai_plotter.save(output_filename, results)
    saved_filenames.append(output_filename)

From saved files, we can create a GIF with the following code:

In [ ]:
from IPython.display import Image

output_filename = f"{output_path}/pad_radar.gif"

GifCreator.execute(output_filename, saved_filenames)

display(Image(output_filename))

---
### <span style='color :blue' > 7. Bias Percentile Plotter</span>  


The “Bias-Percentile” is a proposed visualization to observe the usability of a system considering different Demographic groups. To represent genuine and attacks scores in a percentile graph, use `BiasPercentilePlotter`.

In [ ]:
bias_percentile_plotter = BiasPercentilePlotter(
    title="Bias Percentile",
    demographic=Demographic.SEX,
)

bias_percentile_plotter.show(scores_auxiliary_test)

Use `working_point` tuple to plot a working point region. Imagine you want to compare bias percentiles over a given acceptable working point for a specific use case.

In [ ]:
bias_percentile_plotter = BiasPercentilePlotter(
    title="Bias Percentile",
    demographic=Demographic.SEX,
    working_point=(0.4, 0.6)
)

bias_percentile_plotter.show(scores_auxiliary_test)

###### <span style='color :green' > 7.1 Interactive Bias Percentile Plotter </span>  

Play with the following interactive function:

In [ ]:
@interact(layout=Layout(width='150%', height='80px'))
def calculate_percentile(
    demographic=Demographic.options(),
    lower_wp=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.4),
    higher_wp=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.55),
    approach=Dropdown(
        options=Approach.options(),
        value=Approach.AUXILIARY,
    ),
    protocol=Dropdown(
        options=Protocol.options(),
        value=Protocol.GRANDTEST_SEX_50_50,
    ),
    subset=Dropdown(
        options=Subset.options(),
        value=Subset.TEST,
    )
):
    title = f"Percentile ({demographic.value})"
            
    bias_percentile_plotter = BiasPercentilePlotter(
        title=title,
        demographic=demographic,
        working_point=(lower_wp, higher_wp)
    )
    
    scores = ScoresProvider.get(
        approach, 
        protocol, 
        subset
    )
    bias_percentile_plotter.show(scores)